In [1]:
import pandas as pd
import numpy as np
import haversine as hs
import os, sys
import csv

In [3]:
df = pd.read_csv('link_seoul_ver3.csv', encoding='cp949')
link = df['LINK_ID'].values
geometry = df['geometry']

geometry = geometry.str.replace('LINESTRING', '')
geometry = geometry.str.replace('(', '')
geometry = geometry.str.replace(')', '')
geometry = geometry.str.replace(' ', ',')
geometry = geometry.str.replace(',,', ',')
geometry = geometry.str.split(',')
for i in range(len(geometry)):
    geometry[i].pop(0)


def distance(line_x1, line_y1, line_x2, line_y2, point_x, point_y):
    ab = np.array([line_x2 - line_x1, line_y2 - line_y1])
    ac = np.array([point_x - line_x1, point_y - line_y1])
    result = np.array([line_x1, line_y1]) + ab * (np.dot(ab, ac) / np.dot(ab, ab))
    return hs.haversine((point_x, point_y), (result[0], result[1]))


'''
def distance(x1, y1, x2, y2, x3, y3):
    if x1 == x2:
        m = 0
    else:
        m = (y2 - y1) / (x2 - x1)
    a = m
    b = -1
    c = y1 - m * x1
    d = abs(a * x3 + b * y3 + c) / np.sqrt(a ** 2 + b ** 2)
    return d
'''


def distance_link(link, geometry, x, y):
    d = []
    for i in range(0, len(geometry[link]) - 2, 2):
        x1 = float(geometry[link][i + 1])
        y1 = float(geometry[link][i])
        x2 = float(geometry[link][i + 3])
        y2 = float(geometry[link][i + 2])
        d.append(distance(x1, y1, x2, y2, x, y))
    return min(d)


def distance_haversine(link, geometry, x, y):
    for i in range(len(geometry[link])):
        if i % 2 == 0:
            x1 = float(geometry[link][i + 1])
            y1 = float(geometry[link][i])
            if hs.haversine((x, y), (x1, y1)) <= 0.2:
                return True
            else:
                continue
        else:
            continue


def link_haversine(geometry, x, y):
    link_haversine = []
    for i in range(len(geometry)):
        if distance_haversine(i, geometry, x, y):
            link_haversine.append(i)
    return link_haversine


def link_shortest(link_haversine, geometry, x, y):
    d = []
    for i in range(len(link_haversine)):
        d.append(distance_link(link_haversine[i], geometry, x, y))
    if len(d) == 0:
        return None
    else:
        return link_haversine[d.index(min(d))]


def mapping_link(x, y):
    linked_haversine = link_haversine(geometry, x, y)
    return link_shortest(linked_haversine, geometry, x, y)


def indexToLink(index: int) -> int:
    return int(df.iloc[index]["LINK_ID"])

def save_result(data):
    data_list = []

    for idx in range(len(data)):
        link_id = mapping_link(data.iloc[idx, 4], data.iloc[idx, 3])

        if link_id is None:
            link_id = np.nan

        else:
            link_id = indexToLink(link_id)
        
        data_list.append(link_id)

    return data_list


# loc1 = (37.547871, 126.997193)
# print(mapping_link(loc1[0],loc1[1]))
# 입력은 위경도

C:\Users\User\AppData\Local\Temp\ipykernel_7328\369921849.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  geometry = geometry.str.replace('(', '')
C:\Users\User\AppData\Local\Temp\ipykernel_7328\369921849.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  geometry = geometry.str.replace(')', '')


In [6]:
culture1 = pd.read_csv("POI/문화/공연장.csv", encoding='cp949')
print(culture1)

      Unnamed: 0        좌표정보(x)        좌표정보(y)          경도         위도
0              0  169817.360037  344273.806584  126.660586  36.599128
1              1  149762.965546  364903.326481  126.435112  36.784170
2              2  342040.078340  263570.803333  128.570464  35.862035
3              3  328965.130661  250161.463857  128.423592  35.743026
4              4  343594.783952  265157.612523  128.587956  35.876104
...          ...            ...            ...         ...        ...
3278        3278  192165.934530  446508.000793  126.909279  37.520772
3279        3279  193321.886519  447552.076521  126.922346  37.530188
3280        3280  193019.645823  447560.162310  126.918926  37.530258
3281        3281  193321.886519  447552.076521  126.922346  37.530188
3282        3282  193321.886519  447552.076521  126.922346  37.530188

[3283 rows x 5 columns]


In [ ]:
result = save_result(culture1)
culture1["LINK"] = result
culture1 = culture1.dropna(axis=0)

print(culture1)

In [ ]:
culture1.to_csv("POI/문화/공연장_link.csv")